In [ ]:
from pathlib import Path

import shutil
from tqdm import tqdm
import numpy as np
from datetime import datetime
import pandas as pd

import pytz
tz = pytz.timezone('US/Pacific')

In [ ]:
root  = '/media/naka/3461-6230/DCIM/'

In [ ]:
# root  = '/home/naka/art/timelapse_temp3'

In [ ]:
fps = Path(root).glob('**/*.JPG')
results = []
for f in fps:
    ctime = f.stat().st_ctime
    pt_time = tz.localize(datetime.fromtimestamp(ctime))
    utc_time = pt_time.astimezone(pytz.utc)
    results.append({
        'filename': f.name,
        'filepath': f.as_posix(),
        'path': f,
        'ctime': ctime,
        'mtime': datetime.fromtimestamp(f.stat().st_mtime),
        'created_at_pt': pt_time,
        'created_at_utc': utc_time,
        't_photo': datetime.fromtimestamp(ctime),
    })
    
photo_times = pd.DataFrame(results)

In [ ]:
draw_times = pd.read_csv('/home/naka/art/axidraw_timing/test.csv', index_col=0)
draw_times['t_start'] = pd.to_datetime(draw_times['t_start'])
draw_times['t_end'] = pd.to_datetime(draw_times['t_end'])

draw_times['t_middle'] = draw_times.loc[:, ['t_start', 't_end']].mean(axis=1)
draw_times['duration'] = draw_times['t_end'] - draw_times['t_start']
draw_times['t_middle'] = draw_times['t_start'] + draw_times['duration']/2

draw_times = draw_times.set_index('t_middle')

In [ ]:
photo_times = photo_times.iloc[1::2].reset_index(drop=True)

In [ ]:
offset = np.timedelta64(-1900, 'ms')
closest_photos = []
for ii, row in draw_times.iterrows():
    dtdf = photo_times.copy()
    dtdf['dt'] = (dtdf['t_photo'] - row['t_end']) + offset
    dtdf['abs_dt'] = dtdf['dt'].abs()
    dtdf['close_enough'] = dtdf['dt'].abs() < np.timedelta64(1, 's')
    
    try:
        closest = dtdf.query('close_enough').sort_values('abs_dt').index[0]
        result_row = dtdf.loc[closest]
    except:
        ind = dtdf.sort_values('abs_dt').index[0]
        closest = dtdf.index[ind]
        result_row = dtdf.loc[closest]
        
    result_row = result_row
    result_row.name = ii
    closest_photos.append(result_row)
df = pd.DataFrame(closest_photos)

In [ ]:
target_dir = Path('/home/naka/art/timelapse_temp')

In [ ]:
for p in tqdm(df['path']):
    try:
        shutil.copy(p, target_dir.joinpath(p.name))
    
    except KeyboardInterrupt:
        break
    
    except:
        pass

In [ ]:
shutil.copy(p, target_dir.joinpath(p.name))